In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import json
import os
from glob import glob
from collections import Counter, defaultdict

In [3]:
from typing import List, Dict

import torch
import torch.nn as nn
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EvalPrediction,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

C:\Users\dhrit\Research\sp25\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import Dataset
from transformers import pipeline

In [6]:
class ChangeDetectionDataset(Dataset):
    def __init__(self, root_dir: str, tokenizer, max_length: int = 512):
        """
        root_dir should be e.g. "easy/train" or "hard/validation"
        Expects files: problem-*.txt and truth-problem-*.json
        """
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length

        # find all txt files
        for txt_path in glob(os.path.join(root_dir, "problem-*.txt")):
            base = os.path.splitext(os.path.basename(txt_path))[0]  # e.g. "problem-3"
            json_path = os.path.join(root_dir, f"truth-{base}.json")
            if not os.path.exists(json_path):
                continue

            # read sentences
            with open(txt_path, encoding="utf-8") as f:
                lines = [l.strip() for l in f.readlines() if l.strip()]
            # read labels
            with open(json_path, encoding="utf-8") as f:
                data = json.load(f)
            changes: List[int] = data["changes"]

            # build pairs (sent_i, sent_{i+1})
            for i, label in enumerate(changes):
                if i + 1 < len(lines):
                    self.examples.append({
                        "sent1": lines[i],
                        "sent2": lines[i+1],
                        "label": label,
                    })

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        # tokenizer will handle truncation and padding (padding done in collator)
        enc = self.tokenizer(
            ex["sent1"],
            ex["sent2"],
            truncation=True,
            max_length=self.max_length,
            #padding="max_length",
        )
        enc["labels"] = torch.tensor(ex["label"], dtype=torch.long)
        return enc

In [7]:
# bertweet = AutoModel.from_pretrained()
bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-large", num_labels=2, 
                                                              problem_type="single_label_classification",
                                                              id2label={0: "NEG", 1: "POS"}, label2id={"NEG": 0, "POS": 1},
                                                             )
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large", use_fast=True, 
                                          normalization=True, add_prefix_space=True,)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
train_ds = ChangeDetectionDataset("hard/train", tokenizer)
eval_ds  = ChangeDetectionDataset("hard/validation", tokenizer)

In [8]:
# lst = [train_ds[i]['attention_mask'].count(1) for i in range(20000)]
# plt.hist(lst)

In [10]:
data_collator = DataCollatorWithPadding(
    tokenizer,                # uses the model’s pad-token ID
    pad_to_multiple_of=8,     # keeps tensors tensor-core friendly; optional
    return_tensors="pt"       # default, explicit for clarity
)

In [11]:
training_args = TrainingArguments(
    output_dir="bertweet_large_crossentropy_hard",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,      # ↓ if you OOM; ↑ if you have VRAM
    auto_find_batch_size = True,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,                          # half-precision = 2× speed, 2× memory
    gradient_accumulation_steps=2,      # acts like batch_size double
    logging_steps=50,
    # extra optimization
    dataloader_num_workers=4,
)

In [12]:
trainer = Trainer(
    model=bertweet,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,   # keep this so metrics decode nicely
    data_collator=data_collator   # <-- perfectly OK to omit
)

C:\Users\dhrit\AppData\Local\Temp\ipykernel_20952\3945470312.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()